In [6]:
import os
import glob
import csv
import subprocess

def run_forecasting_single_or_all(
    single_file_path=r'E:\chaosgrandfinale\fnn-master\blk1.txt',
    single_psr_delay=9,
    single_psr_dim=6
):
    """
    If single_file_path is None, process all .txt files in chaotic_folder recursively.
    If single_file_path is not None, process only that .txt file using the given PSR values.
    """

    # Paths
    chaotic_folder = r"E:\forecasting1\forecastingfolder1\chaotic_columns"
    lyapunov_folder = r"E:\forecasting1\forecastingfolder1\lyapunov_result"

    # Base arguments for run.py
    base_command = [
        "python", "run.py",
        "--model_id", "blk_test_96_96",
        "--model", "Attraos",
        "--data", "custom",
        "--features", "S",
        "--enc_in", "1",
        "--dec_in", "1",
        "--c_out", "1",
        "--seq_len", "96",
        "--label_len", "24",
        "--pred_len", "48",
        "--batch_size", "32",
        "--e_layers", "1",
        "--learning_rate", "0.0001",
        "--is_training", "1",
        "--root_path", chaotic_folder,  # common base folder
        "--gpu", "0",
        "--FFT_evolve", "True",
        "--multi_res", "True"
    ]

    # Minimum required number of data points
    min_data_length = 206  # e.g., 110 + 96

    # ---------------------------------------------------------------------
    # Case 1: A single file is specified.
    # ---------------------------------------------------------------------
    if single_file_path is not None:
        if not os.path.isfile(single_file_path):
            print(f"Error: single_file_path does not exist: {single_file_path}")
            return

        # Make sure we have either both or neither. If user explicitly
        # sets single_psr_delay / single_psr_dim, use them; otherwise skip.
        # We'll assume that if user calls single_file_path, they also
        # set single_psr_delay/dim, but you can tweak logic if needed.
        if single_psr_delay is None or single_psr_dim is None:
            print("You specified a single_file_path but did not provide PSR_delay or PSR_dim. "
                  "Either provide them or set single_file_path=None for the all-files flow.")
            return

        # Check length
        try:
            with open(single_file_path, 'r') as f:
                lines = f.readlines()
            if len(lines) < min_data_length:
                print(f"File {single_file_path} is too short (length {len(lines)}); "
                      f"required >= {min_data_length}. Skipping.")
                return
        except Exception as e:
            print(f"Error reading file {single_file_path}: {e}")
            return

        # Derive the team_name from the immediate parent folder
        team_name = os.path.basename(os.path.dirname(single_file_path))

        # data_path relative to chaotic_folder
        relative_data_path = os.path.relpath(single_file_path, chaotic_folder)

        # Build command
        cmd = base_command + [
            "--data_path", relative_data_path,
            "--PSR_delay", str(single_psr_delay),
            "--PSR_dim", str(single_psr_dim),
        ]

        print("\nRunning command (single file):\n", " ".join(cmd), "\n")
        process = subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)

        # Stream stdout line by line
        while True:
            output = process.stdout.readline()
            if output:
                print(output, end='')
            elif process.poll() is not None:
                break

        # Read and print any remaining stderr
        stderr = process.stderr.read()
        if stderr:
            print("===== STDERR =====")
            print(stderr)

        return_code = process.wait()
        print(f"[Process exited with code {return_code}]\n")
        return

    # ---------------------------------------------------------------------
    # Case 2: single_file_path is None -> process ALL .txt files as before.
    # ---------------------------------------------------------------------
    all_txt_files = glob.glob(os.path.join(chaotic_folder, "**", "*.txt"), recursive=True)
    print(f"Found {len(all_txt_files)} .txt files under {chaotic_folder} (recursive).")

    for txt_path in all_txt_files:
        # Parse the folder name (team name) from the immediate parent directory
        team_name = os.path.basename(os.path.dirname(txt_path))

        # Path to the corresponding CSV
        denoised_csv = os.path.join(lyapunov_folder, f"{team_name}.csv")
        if not os.path.isfile(denoised_csv):
            print(f"CSV not found for team {team_name}: {denoised_csv}")
            continue

        # Get the .txt file base name
        txt_filename = os.path.splitext(os.path.basename(txt_path))[0]

        psr_delay = None
        psr_dim = None

        # Open the CSV and find the row
        with open(denoised_csv, mode="r", newline="", encoding="utf-8") as csv_file:
            reader = csv.DictReader(csv_file)
            for row in reader:
                # Case-insensitive match
                if row["Column_Name"].strip().lower() == txt_filename.lower():
                    psr_delay = row["Optimal_Time_Delay"]
                    psr_dim = row["Optimal_Embedding_Dimension"]
                    break

        if not psr_delay or not psr_dim:
            print(f"No matching row found in {denoised_csv} for {txt_filename}")
            continue

        # Check length
        try:
            with open(txt_path, 'r') as f:
                lines = f.readlines()
            if len(lines) < min_data_length:
                print(f"File {txt_path} is too short (length {len(lines)}); required >= {min_data_length}. Skipping.")
                continue
        except Exception as e:
            print(f"Error reading file {txt_path}: {e}")
            continue

        # Compute the relative data_path
        relative_data_path = os.path.relpath(txt_path, chaotic_folder)

        # Construct command
        cmd = base_command + [
            "--data_path", relative_data_path,
            "--PSR_delay", str(psr_delay),
            "--PSR_dim", str(psr_dim),
        ]

        print("\nRunning command:\n", " ".join(cmd), "\n")
        process = subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)

        # Stream stdout line by line
        while True:
            output = process.stdout.readline()
            if output:
                print(output, end='')
            elif process.poll() is not None:
                break

        # Read and print any remaining stderr
        stderr = process.stderr.read()
        if stderr:
            print("===== STDERR =====")
            print(stderr)

        return_code = process.wait()
        print(f"[Process exited with code {return_code}]\n")


In [7]:
# Case A: Process the single .txt file you specified
run_forecasting_single_or_all()



Running command (single file):
 python run.py --model_id blk_test_96_96 --model Attraos --data custom --features S --enc_in 1 --dec_in 1 --c_out 1 --seq_len 96 --label_len 24 --pred_len 48 --batch_size 32 --e_layers 1 --learning_rate 0.0001 --is_training 1 --root_path E:\forecasting1\forecastingfolder1\chaotic_columns --gpu 0 --FFT_evolve True --multi_res True --data_path ..\..\..\chaosgrandfinale\fnn-master\blk1.txt --PSR_delay 9 --PSR_dim 6 

True
Args in experiment:
Basic Config
  Task Name:          long_term_forecast  Is Training:        1                   
  Model ID:           blk_test_96_96      Model:              Attraos             

Data Loader
  Data:               custom              Root Path:          E:\forecasting1\forecastingfolder1\chaotic_columns
  Data Path:          ..\..\..\chaosgrandfinale\fnn-master\blk1.txtFeatures:           S                   
  Target:             OT                  Freq:               h                   
  Checkpoints:        ./check